<h1>Numba-PM-SINTEL-nonmixed-lines</h1><br>


##Configuring notebook if it is run in google colab

In [1]:
colab = False
try:
  from google.colab import drive
  drive.mount("/content/gdrive")
  colab = True
  %cd '/content/gdrive/My Drive/python_projects/FYP'
except Exception:
  pass

##Importing necessary libraries

In [2]:
import numpy as np
from components.classes.NumbaSimpleMatcher import Wrapper as m
from components.classes.SimpleProgressBar import SimpleProgressBar

from components.utils import utils as u
import imp
import cv2
import os
import matplotlib.pyplot as plt
import gc
from components.classes.CSVWriter2 import Wrapper as l
from components.classes.Metrix import Wrapper as me
from components.classes.SintelReader import Wrapper as SintelReader
import math
import time
import glob

##Configuring and testing data generator

In [3]:
ROOT_DATA_SRC = "./optimization/final/"
LOG_DIRECTORY = ROOT_DATA_SRC+"/numba_pm/"
CSV_FILE_NAME =  "pm_normal_images_1024.csv"
CSV_FILE_PATH = os.path.join(LOG_DIRECTORY, CSV_FILE_NAME)


path = "./datasets/sintel/training"
reader = SintelReader(rootPath=path)
#reader.print_available_scenes()
reader.set_selected_scene('cave_4')
loaded_imgs = reader.get_selected_scene_next_files()
log = l(file_path = CSV_FILE_PATH)
progress_bar = SimpleProgressBar.get_instance()

File ./optimization/final//numba_pm/pm_normal_images_1024.csv already exists, header has not been written.


##Matcher visual and metrical testing

> Indented block



##Initialising loop parameters

In [4]:
ORIGINAL_IMG_DIM = loaded_imgs[0].shape
STARTING_ROW_FOR_SLICES = 0
ROW_HEIGHT_PER_SLICE = int(ORIGINAL_IMG_DIM[0]/10)
NUMBER_OF_COLUMNS = int(ORIGINAL_IMG_DIM[1]/2)
SCENES = reader.get_available_scenes()


SLICE_DIM = [ROW_HEIGHT_PER_SLICE, NUMBER_OF_COLUMNS]
main_temp = []
for i in range(10):
  reader.set_selected_scene(SCENES[int(i*2)])
  temp = reader.get_selected_scene_next_sliced(SLICE_DIM, starting_row = 0)
  temp = np.array(temp)
  main_temp.append(temp)



main_temp = np.array(main_temp)
print(main_temp.shape)



spec_dict = dict({})
spec_dict["is_parallel"] = "True"
spec_dict["img_name"] = "sintel_pm_seq_1024"
spec_dict["alg_type"] =  "pm_numba_matcher_avg"
spec_dict["is_img_preprocessed"] = "False"
spec_dict["convolution_filters"] = "-"
spec_dict["filter_strategy"] = "-"
spec_dict["matching_mode"] = non_occ =  "occlusions_added_to_the_groundtruth"
spec_dict["matrix_init_mode"] = 2


loop_params = {}
loop_params["match"] = {"start": 80, "end":50, "step":-10}
loop_params["gap"] = {"start":0, "end":-60, "step":-10}
loop_params["egap"] = {"start":-1, "end":-32, "step":-5}

multiplyers = [2,3,4,5]
number_of_iterations = 5

for v in loop_params.values():
  temp = math.ceil(abs((v["start"]-v["end"])/v["step"]))
  print(temp)
  number_of_iterations *= temp

number_of_iterations =number_of_iterations * len(multiplyers) * main_temp.shape[0]
print("The number of iterations based on the params: {0}".format(round(number_of_iterations)))

(5, 5, 109, 512)
3
6
7
The number of iterations based on the params: 12600


##Running the optimisation

In [5]:
progress_counter = 1
for m_score in range(loop_params["match"]["start"], loop_params["match"]["end"], loop_params["match"]["step"]):
    for gap_score in range(loop_params["gap"]["start"], loop_params["gap"]["end"], loop_params["gap"]["step"]):
        for egap_score in range(loop_params["egap"]["start"], loop_params["egap"]["end"], loop_params["egap"]["step"]):
              for multi in multiplyers:
                for optimizer_img in main_temp:
                  spec_dict["matrix_init_mode"] = 2
                  match = spec_dict["match"] = m_score*multi
                  gap = spec_dict["gap"] = gap_score*multi
                  egap = spec_dict["egap"] = egap_score*multi


                  matcher = m(m_score*multi, gap_score*multi, egap_score*multi)
          
                  matcher.set_images(optimizer_img[1], optimizer_img[0])  
                  matcher.configure_instance()      
                  tic=time.time()

                  x,raw_disp_map = matcher.run_pipeline()
                  toc = time.time()

                  gt_occluded = SintelReader.add_masks_to_raw_disparity(optimizer_img[2], optimizer_img[3], optimizer_img[4])


                  wanna_print = False
                  spec_dict["runtime"] = toc-tic
                  spec_dict["mse"] = me.mse(raw_disp_map, gt_occluded)
                  spec_dict["euclid_distance"] =me.eucledian_distance(raw_disp_map, gt_occluded)
                  spec_dict["ABS_ERR"] = me.avgerr(raw_disp_map, gt_occluded, print=wanna_print, non_occ = True)

                  progress_bar.progress_bar(progress_counter, number_of_iterations, header= "Parameter search progress: ", progress_bar_steps=100)
                  progress_counter+=1
                  log.append_new_sm_results(spec_dict)

UnsupportedParforsError: Failed in nopython mode pipeline (step: nopython mode backend)
[1m[1mThe 'parallel' target is not currently supported on Windows operating systems when using Python 2.7, or on 32 bit hardware.[0m
[0m[1m[1] During: lowering "id=0[LoopNest(index_variable = parfor_index.62, range = (0, $26binary_subscr.10, 1))]{34: <ir.Block at C:\gdrive\python_projects\FYP\components\non_oo_entities\NumbaPatchMatcherFunctions.py (55)>}Var(parfor_index.62, NumbaPatchMatcherFunctions.py:55)" at C:\gdrive\python_projects\FYP\components\non_oo_entities\NumbaPatchMatcherFunctions.py (55)[0m